In [1]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import glob
import os
from skimage.transform import resize
import cv2
import itertools
import warnings
warnings.filterwarnings('ignore')
import skimage as sk
from natsort import natsorted
#torch.set_default_tensor_type('torch.DoubleTensor')

In [2]:
class ConvLSTMCell(nn.Module):
    def __init__(self,class_size, input_size, input_dim, hidden_dim, kernel_size, bias):
        super(ConvLSTMCell, self).__init__()
    
        self.height, self.width = input_size
        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        
        self.padding     = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias        = bias
        self.class_size = class_size
        
        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)
        
        self.linear = nn.Linear(in_features=self.hidden_dim*self.height*self.width,out_features=self.class_size,bias=self.bias)
        
    def forward(self, input_tensor, cur_state):
        
        if cur_state is None:
            cur_state = (Variable(torch.zeros(1, self.hidden_dim, self.height, self.width)),
                Variable(torch.zeros(1, self.hidden_dim, self.height, self.width)))
        
        h_cur, c_cur = cur_state
        
        #print(input_tensor.size(),h_cur.size())
        combined = torch.cat((input_tensor, h_cur), dim=1)  # concatenate along channel axis
        #print(combined.size())
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1) 
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)
        
        next_state = (h_next,c_next)
        
        softmax_out = F.softmax(self.linear(h_next.view(1,-1)),dim=1)
        
        return next_state,softmax_out

conv_lstm_cell = ConvLSTMCell(99,(1,1),512,128,(3,3),True).float()
    

In [3]:
def create_pair_lists(train_test_path):
    
    class_names = [names for names in glob.glob(train_test_path+"/*")]
    #print(class_names)
    
    a = []
    for names in class_names:
        folder = []
        for folders in glob.glob(names+"/*"):
            folder.append(folders)
        a.append(folder)
        

    paired_folders = itertools.zip_longest(*a)
    
    return list(paired_folders)

pair_training = create_pair_lists(train_test_path= "Frames/Train")
#pair_training[0]

In [4]:
def npy_list(folder):
    npy_list = []
    for npy_file in glob.glob(folder+"/*.npy"):
        npy_list.append(npy_file)
    npy_list = natsorted(npy_list)
    
    return npy_list

In [5]:
def one_hot(array, num_classes):
    return np.squeeze(np.eye(num_classes)[array.reshape(-1)])

In [6]:
def concatenate_npy(npy_list):
    X = []
    for npys in npy_list:
        array = np.load(npys)
        X.append(array)
    return np.array(X)

In [7]:
def array_to_tensor(array):
    tensor = torch.from_numpy(array).float()
    return tensor

In [15]:
def evaluate(out,real):
    #print(out.size())
    #print(real.size())
    #print(out.size(),real.size())
    log_loss = torch.mean(-real*torch.log(out))
    mean_loss = torch.mean((out-real)**2)
    
    loss = (mean_loss + log_loss)/2.0
    
    real_arg = torch.argmax(real,dim=1)
    out_arg = torch.argmax(out,dim=1)
    #print(sum(out_arg==real_arg).item())
    correct = sum(out_arg==real_arg).item()
    return log_loss,correct

In [ ]:
npy = npy_list(pair_training[0][1])

In [27]:
def train_lstm(nb_epoch,model,lr):
    
    optimizer = torch.optim.RMSprop(model.parameters(),lr=lr)
    
    nb_classes = 99
    for epoch in range(1,nb_epoch+1):

        train_epoch_loss = 0.0
        test_epoch_loss = 0.0
        pair_nu = 1

        for train_pairs in pair_training:
            
            label = np.array([0])
            for folders in train_pairs:            
                print(folders)
                Y = array_to_tensor(one_hot(label,nb_classes)).view(1,-1)
        
                if folders is not None:
                
                    np_list = npy_list(folders)
                    X = concatenate_npy(np_list)
                    X = array_to_tensor(X)

                    state = None
                    for f in range(0,X.size()[0]):
                        inp = X[f:f+1]
                        state,softmax_out = conv_lstm_cell(inp,state)
                        #print(inp.size(),state[0].size(),softmax_out.size())

                    last_hidden = state[0]
                    last_softmax = softmax_out
                    
                    loss,correct = evaluate(last_softmax,Y)
                    
                    print("loss: "+str(loss.item()))
                    print(torch.argmax(last_softmax,dim=1).item())
                    print(torch.argmax(Y,dim=1).item())
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                print("--------------")
                
                label += 1
                    
                
                
                
                
                
                
                
    

In [ ]:
train_lstm(1,conv_lstm_cell,1.e-2)

In [ ]:
a = np.array([1,3,2,0])
r = np.array([[0.2,0.3,0.4,0.1],[0.05,0.25,0.1,0.7],[0.3,0.35,0.2,0.15],[0.9,0.05,0.01,0.04]])
on = one_hot(a,4)
r

In [ ]:
np.argmax(on,1)

In [ ]:
on 

In [ ]:
r

In [ ]:
for e in range(r.shape[0]):
    row = r[e,:]
    idx = (-row).argsort()[:2]
    print(idx)

In [ ]:
o = [0]
k = np.array(o)

In [ ]:
k + 1

In [ ]:
label = np.array([0])
for k in range(5):
    print(one_hot(label,5))
    label += 1
    